Working jupyter to create function to create a list of aminoacids likely to be used for mutation

In [13]:
# import packages
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt


Function finished

In [12]:
def aa_mut_select(input_path, pdb_file, output_path, df=False):
    from function import salt_bridge
    from function import H_bond_calc
    from function import VdW_interaction
    from function import pdb2pqr
    from helper_function import remove_nan
    import os
    import numpy as np
    import pandas as pd
    prot_name = pdb_file.split('-')[1]
    pqr_file = f'{(pdb_file.split('.')[0]).split('-')[1]}.pqr'
    
    #create pqr file
    if os.path.isfile(os.path.join(output_path, f'{(pdb_file.split(".")[0]).split("-")[1]}.pqr')):
        print('Pqr file already exists')
    else:    
        pdb2pqr(input_path, output_path, pdb_file)
    

    # get atom features
    Salt_bridge = salt_bridge(input_path, pdb_file)
    print('Salt_bridge finished')
    H_bond = H_bond_calc(output_path, pqr_file)
    print('H_bond finished')
    VdW_clust, VdW_vol = VdW_interaction(input_path, pdb_file, by_atom = True)
    print('VdW_interaction finished')
    
    # extract the values for the proteins from the dictionary and delete atoms that dont have a feature (if applicable)
    Salt_bridge = remove_nan(Salt_bridge[prot_name])
    H_bond = remove_nan(H_bond[prot_name][:,:,0])

    VdW_clust = VdW_clust[prot_name]
 

    
    
    #create lists with all aminoacid that are part of a feature
    atom_S =list(Salt_bridge[0,1:])
    atom_HA = list(H_bond[0,1:])
    atom_HD = list(H_bond[1:,0])

    
    
    # creates an atom_dict that contains the atom number and the feature it is part of
    atom_dict = {}
    for lst, identifier in [(atom_S, "Salt_bridge"), (atom_HA, "Hbond_acc"), (atom_HD, "Hbond_don")]:
        for atom_number in lst:
            if atom_number in atom_dict:
                atom_dict[atom_number] = [atom_dict[atom_number], identifier]
            else:
                atom_dict[atom_number] = identifier
    for k,v in VdW_clust.items():
        if k in atom_dict:
            atom_dict[k] = [atom_dict[k], v]
        else: atom_dict[k] = v
    atom_sorted = {k: atom_dict[k] for k in sorted(atom_dict)}


    # create a dataframe with the atom number and the feature it is part of
    prot_df = pd.DataFrame(columns = ['Protein','Aminoacid','Aminoacid_number', 'Atom_number', 'Feature'])
    Protein_array = np.empty((0, 5))
    with open (os.path.join(output_path, pqr_file)) as f:
        prot_df_list = []
        for line in f:
            line = line.replace('-', '  -')
            if line.startswith('ATOM'):
                atom_number = int(line.split()[1])
                feature = atom_sorted.get(atom_number)
                if atom_number in atom_sorted.keys():
                    atom_line = np.array([[str(prot_name),str(line.split()[3]), float(line.split()[4]), float(line.split()[1]), str(feature)]])
                    Protein_array = np.append(Protein_array, atom_line, axis=0)
    
    if df:
        Prot_df = pd.DataFrame(Protein_array, columns = ['Protein','Aminoacid','Aminoacid_number', 'Atom_number', 'Feature'])
         return Prot_df
    else:
        return Protein_array


In [13]:
path_pdb = './data/pdbs'
path_pqr=  './data/pqrs'
pdb_file = 'AF-C0SP86-F1.pdb'
pqr_file = 'P39846.pqr'
output_path = './data/test'
name = pqr_file.split('.')[0]


prot_df = aa_mut_select(path_pdb, pdb_file, output_path)

Pqr file already exists
Salt_bridge finished
H_bond finished


c:\Users\marik\OneDrive - bwedu\Uni HD\FS 4\Bioinfo\topic04_02\helper_function.py:71: RuntimeWarning: invalid value encountered in cast
  distance[:,0] = distance[:,0].astype('int')
c:\Users\marik\OneDrive - bwedu\Uni HD\FS 4\Bioinfo\topic04_02\helper_function.py:89: RuntimeWarning: invalid value encountered in arccos
  theta = np.arccos((d_DH[n,1,1]**2 + d_HA[n,1:]**2 - d_DA[1:,1:][n,:]**2)/(2*d_DH[n,1,1]*d_HA[1:,1:][n,:]))


VdW_interaction finished


c:\Users\marik\OneDrive - bwedu\Uni HD\FS 4\Bioinfo\topic04_02\helper_function.py:106: RuntimeWarning: invalid value encountered in cast
  array[:,0] = array[:,0].astype('int')


In [14]:
print(prot_df)




    Protein Aminoacid Aminoacid_number Atom_number    Feature
0    C0SP86       SER              2.0        29.0  Cluster 0
1    C0SP86       TRP              3.0        31.0  Cluster 0
2    C0SP86       TRP              3.0        33.0  Cluster 0
3    C0SP86       HIS              5.0        86.0  Cluster 1
4    C0SP86       HIS              5.0        88.0  Cluster 1
..      ...       ...              ...         ...        ...
648  C0SP86       ARG            920.0     14403.0  Hbond_don
649  C0SP86       GLU            926.0     14511.0  Hbond_acc
650  C0SP86       LYS            935.0     14630.0  Hbond_don
651  C0SP86       ARG            940.0     14707.0  Hbond_don
652  C0SP86       GLU            941.0     14730.0  Hbond_acc

[653 rows x 5 columns]
